In [93]:
%reset
import midi
import os
import numpy as np



Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [74]:

os.chdir('C:\\Users\\Paul\\Neural_Networks\\Project\\Generating_Music\\Midi_Files')
print(os.getcwd())

C:\Users\Paul\Neural_Networks\Project\Generating_Music\Midi_Files


In [95]:
lowerBound = 24
upperBound = 102

def midiToNoteStateMatrix(midifile):

    pattern = midi.read_midifile(midifile)

    timeleft = [track[0].tick for track in pattern]

    posns = [0 for track in pattern]

    statematrix = []
    span = upperBound-lowerBound
    time = 0

    state = [[0,0] for x in range(span)]
    statematrix.append(state)
    while True:
        if time % (pattern.resolution / 4) == (pattern.resolution / 8):
            # Crossed a note boundary. Create a new state, defaulting to holding notes
            oldstate = state
            state = [[oldstate[x][0],0] for x in range(span)]
            statematrix.append(state)

        for i in range(len(timeleft)):
            while timeleft[i] == 0:
                track = pattern[i]
                pos = posns[i]

                evt = track[pos]
                if isinstance(evt, midi.NoteEvent):
                    if (evt.pitch < lowerBound) or (evt.pitch >= upperBound):
                        pass
                        # print "Note {} at time {} out of bounds (ignoring)".format(evt.pitch, time)
                    else:
                        if isinstance(evt, midi.NoteOffEvent) or evt.velocity == 0:
                            state[evt.pitch-lowerBound] = [0, 0]
                        else:
                            state[evt.pitch-lowerBound] = [1, 1]
                elif isinstance(evt, midi.TimeSignatureEvent):
                    if evt.numerator not in (2, 4):
                        # We don't want to worry about non-4 time signatures. Bail early!
                        # print "Found time signature event {}. Bailing!".format(evt)
                        return statematrix

                try:
                    timeleft[i] = track[pos + 1].tick
                    posns[i] += 1
                except IndexError:
                    timeleft[i] = None

            if timeleft[i] is not None:
                timeleft[i] -= 1

        if all(t is None for t in timeleft):
            break

        time += 1

    return statematrix

In [115]:
midifile = 'chpn_op7_1.mid'
NSB = midiToNoteStateMatrix(midifile)

pattern = midi.read_midifile(midifile)
print(pattern)



midi.Pattern(format=1, resolution=480, tracks=\
[midi.Track(\
  [midi.TrackNameEvent(tick=0, text='Chopin Mazurka Opus 7 No. 1', data=[67, 104, 111, 112, 105, 110, 32, 77, 97, 122, 117, 114, 107, 97, 32, 79, 112, 117, 115, 32, 55, 32, 78, 111, 46, 32, 49]),
   midi.CopyrightMetaEvent(tick=0, text='Copyright \xa9 2003 by Bernd Krueger ', data=[67, 111, 112, 121, 114, 105, 103, 104, 116, 32, 169, 32, 50, 48, 48, 51, 32, 98, 121, 32, 66, 101, 114, 110, 100, 32, 75, 114, 117, 101, 103, 101, 114, 32]),
   midi.TextMetaEvent(tick=0, text='Frederic Chopin', data=[70, 114, 101, 100, 101, 114, 105, 99, 32, 67, 104, 111, 112, 105, 110]),
   midi.TextMetaEvent(tick=0, text='Vivace', data=[86, 105, 118, 97, 99, 101]),
   midi.TextMetaEvent(tick=0, text='Fertiggestellt am 3.7.2003\n', data=[70, 101, 114, 116, 105, 103, 103, 101, 115, 116, 101, 108, 108, 116, 32, 97, 109, 32, 51, 46, 55, 46, 50, 48, 48, 51, 10]),
   midi.TextMetaEvent(tick=0, text='Update am 4.7.2010\n', data=[85, 112, 100, 97, 116,